In [1]:
import os
import re
import string
import numpy as np
import pandas as pd
import mlflow
import mlflow.sklearn
import dagshub
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import warnings
warnings.simplefilter("ignore", UserWarning)
warnings.filterwarnings("ignore")

In [2]:
MLFLOW_TRACKING_URI = "https://dagshub.com/SoumilMalik24/Sentiment-Analysis-with-MLOps.mlflow"
dagshub.init(repo_owner="SoumilMalik24", repo_name="Sentiment-Analysis-with-MLOps", mlflow=True)
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment("LoR Hyperparameter Tuning")

Accessing as SoumilMalik24

Initialized MLflow to track repo "SoumilMalik24/Sentiment-Analysis-with-MLOps"

Repository SoumilMalik24/Sentiment-Analysis-with-MLOps initialized!

2026/01/22 10:51:36 INFO mlflow.tracking.fluent: Experiment with name 'LoR Hyperparameter Tuning' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/cb624a1f28034acdbc7ca181133ae458', creation_time=1769059296591, experiment_id='2', last_update_time=1769059296591, lifecycle_stage='active', name='LoR Hyperparameter Tuning', tags={}>

In [3]:
def preprocess_text(text):
    """Applies multiple text preprocessing steps."""
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words("english"))

    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = re.sub(f"[{re.escape(string.punctuation)}]", " ", text)  # Remove punctuation
    text = re.sub(r'https?://\S+|www\.\S+', '', text)  # Remove URLs
    text = " ".join([lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words])  # Lemmatization & stopwords removal
    
    return text.strip()

In [4]:
def load_and_prepare_data(filepath):
    """Loads, preprocesses, and vectorizes the dataset."""
    df = pd.read_csv(filepath)
    
    # Apply text preprocessing
    df["review"] = df["review"].astype(str).apply(preprocess_text)
    
    # Filter for binary classification
    df = df[df["sentiment"].isin(["positive", "negative"])]
    df["sentiment"] = df["sentiment"].map({"negative": 0, "positive": 1})
    
    # Convert text data to TF-IDF vectors
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(df["review"])
    y = df["sentiment"]
    
    return train_test_split(X, y, test_size=0.2, random_state=42), vectorizer

In [5]:
def train_and_log_model(X_train, X_test, y_train, y_test, vectorizer):
    """Trains a Logistic Regression model with GridSearch and logs results to MLflow."""
    
    param_grid = {
        "C": [0.1, 1, 10],
        "penalty": ["l1", "l2"],
        "solver": ["liblinear"]
    }
    
    with mlflow.start_run():
        grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5, scoring="f1", n_jobs=-1)
        grid_search.fit(X_train, y_train)

        # Log all hyperparameter tuning runs
        for params, mean_score, std_score in zip(grid_search.cv_results_["params"], 
                                                 grid_search.cv_results_["mean_test_score"], 
                                                 grid_search.cv_results_["std_test_score"]):
            with mlflow.start_run(run_name=f"LR with params: {params}", nested=True):
                model = LogisticRegression(**params)
                model.fit(X_train, y_train)
                
                y_pred = model.predict(X_test)
                
                metrics = {
                    "accuracy": accuracy_score(y_test, y_pred),
                    "precision": precision_score(y_test, y_pred),
                    "recall": recall_score(y_test, y_pred),
                    "f1_score": f1_score(y_test, y_pred),
                    "mean_cv_score": mean_score,
                    "std_cv_score": std_score
                }
                
                # Log parameters & metrics
                mlflow.log_params(params)
                mlflow.log_metrics(metrics)
                
                print(f"Params: {params} | Accuracy: {metrics['accuracy']:.4f} | F1: {metrics['f1_score']:.4f}")

        # Log the best model
        best_params = grid_search.best_params_
        best_model = grid_search.best_estimator_
        best_f1 = grid_search.best_score_

        mlflow.log_params(best_params)
        mlflow.log_metric("best_f1_score", best_f1)
        mlflow.sklearn.log_model(best_model, "model")
        
        print(f"\nBest Params: {best_params} | Best F1 Score: {best_f1:.4f}")


In [7]:
(X_train, X_test, y_train, y_test), vectorizer = load_and_prepare_data("data.csv")

train_and_log_model(X_train, X_test, y_train, y_test, vectorizer)

Params: {'C': 0.1, 'penalty': 'l1', 'solver': 'liblinear'} | Accuracy: 0.5400 | F1: 0.0000
🏃 View run LR with params: {'C': 0.1, 'penalty': 'l1', 'solver': 'liblinear'} at: https://dagshub.com/SoumilMalik24/Sentiment-Analysis-with-MLOps.mlflow/#/experiments/2/runs/8a9c9a64a47642dfb3b91216884b933e
🧪 View experiment at: https://dagshub.com/SoumilMalik24/Sentiment-Analysis-with-MLOps.mlflow/#/experiments/2
Params: {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'} | Accuracy: 0.5400 | F1: 0.0000
🏃 View run LR with params: {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'} at: https://dagshub.com/SoumilMalik24/Sentiment-Analysis-with-MLOps.mlflow/#/experiments/2/runs/6729ca506c6a497eb550c0572d276e23
🧪 View experiment at: https://dagshub.com/SoumilMalik24/Sentiment-Analysis-with-MLOps.mlflow/#/experiments/2
Params: {'C': 1, 'penalty': 'l1', 'solver': 'liblinear'} | Accuracy: 0.5800 | F1: 0.4324
🏃 View run LR with params: {'C': 1, 'penalty': 'l1', 'solver': 'liblinear'} at: https://dagshub.c

2026/01/22 10:52:34 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/01/22 10:52:44 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.



Best Params: {'C': 10, 'penalty': 'l2', 'solver': 'liblinear'} | Best F1 Score: 0.6706
🏃 View run honorable-quail-535 at: https://dagshub.com/SoumilMalik24/Sentiment-Analysis-with-MLOps.mlflow/#/experiments/2/runs/00e4ea8b229340608bb1af883fdfebe8
🧪 View experiment at: https://dagshub.com/SoumilMalik24/Sentiment-Analysis-with-MLOps.mlflow/#/experiments/2
